In [1]:
import pandas as pd
from datetime import datetime

pd.options.display.max_rows=500

In [2]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

In [20]:
class USDataCleanUp():
    """
    Take in a dataset, clean up the US data that has changed format,
    and then aggregate back into main dataset
    """
    def __init__(self, df, key_col):
        self.original_df = df
        #self.cleaned_us_data = pd.DataFrame()
        self.data = pd.DataFrame()
        self.key_col = key_col

    def setup_US_data(self):
        """
        Add a city and state column for old historical data
        Then split between old data format and new
        """
        df = self.original_df.copy()
        
        # Limit to US only
        df = df.loc[df['country_region'] =='US']
        
        # Split province and state by comma into 2 columns (city and state)
        df = pd.concat([df, df['province_state'].str.split(', ', expand=True)], axis=1)
        df = df.rename(columns={0:'city',
                  1:'state'})
        
        # Add a state abbreviation for new US data 
        df['US_new_data_state_abbrev'] = df['province_state'].map(us_state_abbrev)
        
        # The Diamond/Grand princess are in here but NOT states
        df['state_cleaned'] = df['US_new_data_state_abbrev'].combine_first(df['state']).combine_first(df['province_state'])

        # DC comes thru as DC and D.C.
        df['state_cleaned'] = df.state_cleaned.str.replace('.','')
        
        self.JEFF_TEST_DELETE = df
        # Split between old and new
        self.US_old = df.loc[df['date'] <= datetime(2020, 3, 9)]
        self.US_new = df.loc[df['date'] > datetime(2020, 3, 9)]
        
        
    def handle_old_data(self):
        """
        Logic to aggregate old data
        """
        old_df = self.US_old.groupby(['state_cleaned','date'])[self.key_col].sum().reset_index()
        return old_df
    
    def handle_new_data(self):
        """
        Logic to aggregate new data
        """
        new_df = self.US_new.groupby(['state_cleaned','date'])[self.key_col].sum().reset_index()
        #new_df = self.US_new[['state_cleaned','date','confirmed_cases']]
        return new_df
    
    def combine_US_data(self):
        """
        Combine old and new US data
        """
        df = pd.concat([self.handle_old_data(),
                                self.handle_new_data()])
        df['country_region'] = 'US'
        df = df.rename(columns={'state_cleaned':'province_state'})
        
        return df
        
    def prepare_final_cleaned_data(self):
        df1 = self.original_df
        df2 = self.combine_US_data()

        # Remove old US data
        df1 = df1.loc[df1['country_region'] != 'US']

        # Combine old data with cleaned US
        self.data = pd.concat([df1,df2],
                               axis=0)        

    def run(self):
        self.setup_US_data()
        self.combine_US_data()   
        self.prepare_final_cleaned_data()

In [33]:
class HopkinsDataCleaner(object):
    """
    Accepts a raw URL from Hopkins GitHub and formats the data 
    to be fed into Power BI
    """
    def __init__(self, url, dataset_name):
        self.url = url
        self.dataset_name = dataset_name
        self.col_confirmed = f"confirmed_{self.dataset_name}"
        self.col_confirmed_prev_day = f"{self.col_confirmed}_prev_day"
        self.col_daily_diff = f"daily_diff_{self.dataset_name}"
        
    def read_initial_data(self):
        """Read data from URL"""
        # Read and clean up column headers
        JH_df = pd.read_csv(self.url)
        JH_df.columns = JH_df.columns.str.lower().str.replace(' ','_')
        JH_df = JH_df.rename(columns={'province/state':'province_state',
                               'country/region':'country_region'})
        self.data = JH_df
        
    def stack_initial_dataset(self):
        """
        Want the data in a stacked format
        """
        # Stack the dataset
        df = self.data.copy()
        
        index_cols = ['province_state', 'country_region', 'lat', 'long',]
        df = df.set_index(index_cols).stack().reset_index(name=self.col_confirmed)
        df = df.rename(columns={'level_4':'date'})
        df['date'] = pd.to_datetime(df['date'])

        self.data = df
    
    def handle_US_bad_data(self):
        """
        Pass data to a separate class that can
        clean up the bad US data
        """
        # Clean up column values
        # This class here will handle cleaning up the US porition of the data
        US_cleaner = USDataCleanUp(df=self.data,
                                   key_col=self.col_confirmed)
        US_cleaner.run()
        self.data = US_cleaner.data.copy()
        
    def clean_mid(self):
        """
        Some clean up we need to do AND add a RANK field
        """
        df = self.data.copy()
        # Fill blanks and concat country and state
        df['province_state'] = df['province_state'].fillna('Not Provided')
        df['country_region'] = df['country_region'].fillna('Not Provided')

        # Rank order dates by state/country
        df['state_and_country'] = df['province_state'] + "-" +  df['country_region']
        df['rank'] = df.groupby(['state_and_country'])['date'].rank(ascending=True)
        
        self.data = df

    def create_daily_diff_col(self):
        """
        Here we create a daily difference column
        """
        df = self.data.copy()
        
        # Prep dataset 1-day delta to get day-over-day change
        df['previous_days_date'] = df['date'] + pd.np.timedelta64(1, 'D')
        previous_df = df.copy()
        previous_df = previous_df[['province_state','country_region','previous_days_date',self.col_confirmed]]
        previous_df = previous_df.rename(columns={self.col_confirmed:self.col_confirmed_prev_day})

        # Join dataset to itself with a one day offset to get daily diff
        df = df.merge(previous_df,
                how='left',
                left_on=['province_state','country_region','date'],
                right_on=['province_state','country_region','previous_days_date'])
        
        df = df.drop(['previous_days_date_x','previous_days_date_y'], axis=1)      

        df[self.col_confirmed_prev_day] = df[self.col_confirmed_prev_day].fillna(0)
        df[self.col_daily_diff] = df[self.col_confirmed] - df[self.col_confirmed_prev_day]
        
        self.data = df
        
    def run(self):
        """
        Main run function to execute logic
        """
        self.read_initial_data()
        self.stack_initial_dataset()
        self.handle_US_bad_data()
        self.clean_mid()
        self.create_daily_diff_col()

In [34]:
url_confirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
url_recovered = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'
url_deaths = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'

In [35]:
confirmed = HopkinsDataCleaner(url=url_confirmed,
                              dataset_name='cases')
recovered = HopkinsDataCleaner(url=url_recovered,
                              dataset_name='recoveries')
deaths = HopkinsDataCleaner(url=url_deaths,
                           dataset_name='deaths')

In [36]:
confirmed.run()
recovered.run()
deaths.run()

/Users/amlvt225/anaconda3/envs/osh_v2/lib/python3.7/site-packages/ipykernel_launcher.py:77: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [55]:
df_main = confirmed.data

In [56]:
df_recovered = recovered.data

In [57]:
df_deaths = deaths.data

In [58]:
df_main.head(2)

,confirmed_cases,country_region,date,lat,long,province_state,state_and_country,rank,confirmed_cases_prev_day,daily_diff_cases
0,2,Thailand,2020-01-22,15.0,101.0,Not Provided,Not Provided-Thailand,1.0,0.0,2.0
1,3,Thailand,2020-01-23,15.0,101.0,Not Provided,Not Provided-Thailand,2.0,2.0,1.0


In [59]:
df_recovered.head(2)

,confirmed_recoveries,country_region,date,lat,long,province_state,state_and_country,rank,confirmed_recoveries_prev_day,daily_diff_recoveries
0,0,Thailand,2020-01-22,15.0,101.0,Not Provided,Not Provided-Thailand,1.0,0.0,0.0
1,0,Thailand,2020-01-23,15.0,101.0,Not Provided,Not Provided-Thailand,2.0,0.0,0.0


In [60]:
df_recovered = df_recovered[['confirmed_recoveries', 
              'date', 
              'state_and_country', 
              'confirmed_recoveries_prev_day', 
              'daily_diff_recoveries']]
df_deaths = df_deaths[['confirmed_deaths', 
              'date', 
              'state_and_country', 
              'confirmed_deaths_prev_day', 
              'daily_diff_deaths']]

In [61]:
df_main.shape

(12012, 10)

In [62]:
df_main = df_main.merge(df_recovered,
             how='left',
             on=['date','state_and_country'])

In [63]:
df_main = df_main.merge(df_deaths,
             how='left',
             on=['date','state_and_country'])

In [64]:
df_main.shape

(12012, 16)

In [65]:
df_main

,confirmed_cases,country_region,date,lat,long,province_state,state_and_country,rank,confirmed_cases_prev_day,daily_diff_cases,confirmed_recoveries,confirmed_recoveries_prev_day,daily_diff_recoveries,confirmed_deaths,confirmed_deaths_prev_day,daily_diff_deaths
0,2,Thailand,2020-01-22,15.0,101.0,Not Provided,Not Provided-Thailand,1.0,0.0,2.0,0,0.0,0.0,0,0.0,0.0
1,3,Thailand,2020-01-23,15.0,101.0,Not Provided,Not Provided-Thailand,2.0,2.0,1.0,0,0.0,0.0,0,0.0,0.0
2,5,Thailand,2020-01-24,15.0,101.0,Not Provided,Not Provided-Thailand,3.0,3.0,2.0,0,0.0,0.0,0,0.0,0.0
3,7,Thailand,2020-01-25,15.0,101.0,Not Provided,Not Provided-Thailand,4.0,5.0,2.0,0,0.0,0.0,0,0.0,0.0
4,8,Thailand,2020-01-26,15.0,101.0,Not Provided,Not Provided-Thailand,5.0,7.0,1.0,2,0.0,2.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12007,0,US,2020-03-13,NaN,NaN,WV,WV-US,52.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
12008,0,US,2020-03-10,NaN,NaN,WY,WY-US,49.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
12009,0,US,2020-03-11,NaN,NaN,WY,WY-US,50.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
12010,1,US,2020-03-12,NaN,NaN,WY,WY-US,51.0,0.0,1.0,0,0.0,0.0,0,0.0,0.0


In [66]:
max_date = df_main['date'].max()

In [75]:
yesterday = df_main.loc[df_main['date'] == max_date]
yesterday['confirmed_cases'].sum(), yesterday['confirmed_recoveries'].sum(), yesterday['confirmed_deaths'].sum()

(145193, 70251, 5404)

In [76]:
df_main['daily_diff_cases'].sum(), df_main['daily_diff_recoveries'].sum(), df_main['daily_diff_deaths'].sum()

(145193.0, 70251.0, 5404.0)

In [79]:
df_main.loc[df_main['province_state'] == 'WA'].tail(5)

,confirmed_cases,country_region,date,lat,long,province_state,state_and_country,rank,confirmed_cases_prev_day,daily_diff_cases,confirmed_recoveries,confirmed_recoveries_prev_day,daily_diff_recoveries,confirmed_deaths,confirmed_deaths_prev_day,daily_diff_deaths
11651,122,US,2020-03-09,NaN,NaN,WA,WA-US,48.0,122.0,0.0,1,1.0,0.0,19,18.0,1.0
11996,267,US,2020-03-10,NaN,NaN,WA,WA-US,49.0,122.0,145.0,1,1.0,0.0,23,19.0,4.0
11997,366,US,2020-03-11,NaN,NaN,WA,WA-US,50.0,267.0,99.0,1,1.0,0.0,29,23.0,6.0
11998,442,US,2020-03-12,NaN,NaN,WA,WA-US,51.0,366.0,76.0,1,1.0,0.0,31,29.0,2.0
11999,568,US,2020-03-13,NaN,NaN,WA,WA-US,52.0,442.0,126.0,1,1.0,0.0,37,31.0,6.0


In [80]:
df_main.loc[df_main['province_state'] == 'VA'].tail(5)

,confirmed_cases,country_region,date,lat,long,province_state,state_and_country,rank,confirmed_cases_prev_day,daily_diff_cases,confirmed_recoveries,confirmed_recoveries_prev_day,daily_diff_recoveries,confirmed_deaths,confirmed_deaths_prev_day,daily_diff_deaths
11555,2,US,2020-03-09,NaN,NaN,VA,VA-US,48.0,2.0,0.0,0,0.0,0.0,0,0.0,0.0
11988,7,US,2020-03-10,NaN,NaN,VA,VA-US,49.0,2.0,5.0,0,0.0,0.0,0,0.0,0.0
11989,9,US,2020-03-11,NaN,NaN,VA,VA-US,50.0,7.0,2.0,0,0.0,0.0,0,0.0,0.0
11990,17,US,2020-03-12,NaN,NaN,VA,VA-US,51.0,9.0,8.0,0,0.0,0.0,0,0.0,0.0
11991,30,US,2020-03-13,NaN,NaN,VA,VA-US,52.0,17.0,13.0,0,0.0,0.0,0,0.0,0.0


In [81]:
df_main.loc[df_main['province_state'] == 'CA'].tail(5)

,confirmed_cases,country_region,date,lat,long,province_state,state_and_country,rank,confirmed_cases_prev_day,daily_diff_cases,confirmed_recoveries,confirmed_recoveries_prev_day,daily_diff_recoveries,confirmed_deaths,confirmed_deaths_prev_day,daily_diff_deaths
9443,101,US,2020-03-09,NaN,NaN,CA,CA-US,48.0,95.0,6.0,2,2.0,0.0,1,1.0,0.0
11812,144,US,2020-03-10,NaN,NaN,CA,CA-US,49.0,101.0,43.0,2,2.0,0.0,2,1.0,1.0
11813,177,US,2020-03-11,NaN,NaN,CA,CA-US,50.0,144.0,33.0,2,2.0,0.0,3,2.0,1.0
11814,221,US,2020-03-12,NaN,NaN,CA,CA-US,51.0,177.0,44.0,6,2.0,4.0,4,3.0,1.0
11815,282,US,2020-03-13,NaN,NaN,CA,CA-US,52.0,221.0,61.0,6,6.0,0.0,4,4.0,0.0


## QA of Data 

In [ ]:
JH_df_main.loc[JH_df_main['province_state'].isnull()]['country_region'].value_counts().reset_index()

In [ ]:
# This indicates a shift in the data
df_main.loc[df_main['daily_diff'] < 0]['date'].value_counts()

In [ ]:
# This indicates a shift in the data
df_main.loc[df_main['daily_diff'] < 0]['province_state'].value_counts()

In [ ]:
df_main.loc[df_main['daily_diff'] < 0]

In [ ]:
df_main.loc[(df_main['daily_diff'] < 0) &
      (df_main['province_state'] == 'CA')]

In [ ]:
df_main.loc[df_main['country_region'] == 'China'].head(200).sort_values('rank')

In [ ]:
df_main.country_region.value_counts()

In [82]:
df_main.to_csv('HOPKINS_CLEANED.csv', index=False)